In [1]:
#Mount Drive

In [1]:
!pip install tensorflow==1.14.0
!pip install keras==2.2.0
#After running this restart the notebook and don't run this cell again after restart

     |████████████████████████████████| 109.2MB 94kB/s 
     |████████████████████████████████| 3.2MB 32.0MB/s 
     |████████████████████████████████| 491kB 40.6MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 307kB 2.9MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
ERROR: tensorflow 1.14.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
  Found existing installation: 

In [27]:
import tensorflow as tf
from tensorflow import keras
# cd where dataset is present

In [28]:
cd /content/drive/My Drive/FYP2

/content/drive/My Drive/FYP2


In [29]:
import numpy as np
import os
import cv2
import pandas as pd
import scipy.io as sio
from keras.optimizers import SGD, Adagrad
import keras.backend as K
import tensorflow as tf
import scipy.io as sio
from tensorflow.keras.models import Model
from keras.regularizers import l2
#from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Input, Concatenate, LeakyReLU, Dropout
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
import numpy as np
from random import shuffle
import os
import glob
import copy
import numpy as np
from keras.layers.core import Dense, Dropout
from keras.models import Sequential,load_model
from keras.optimizers import Adagrad, RMSprop
from keras.regularizers import l2
import matplotlib.pyplot as plt
import PIL
from copy import deepcopy
from collections import OrderedDict
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from keras.regularizers import l2
#from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Input, Concatenate, LeakyReLU, Dropout
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam, Adadelta
from keras.callbacks import Callback, ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras_applications import vgg16

In [5]:
base_model=DenseNet121(weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
34119680/34115984 [==============================] - 2s 0us/step


In [6]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [8]:
#Change this according to your data managment
a=['Lemon','Orange','Pear','Apple']
counter=0
count=0
for i in a:
  aa=glob.glob('Test/Fake/'+i+"/*.npy")#Train Images Path
  for j in aa:
    count=count+1
    img = np.load(j)
    res = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    dim = np.zeros((224,224))
    res = np.stack((res,dim, dim), axis=2)
    img_data = np.expand_dims(res, axis=0)
    img_data = preprocess_input(img_data)
    feature = model.predict(img_data)
    np.save("Test_Features/"+i+"/Fake/"+str(count),feature) #Train Features Path
  count=0
  counter=counter+1
  print(counter)

1
2
3
4


In [20]:
a=['Lemon','Orange','Pear','Apple']
b=list()
for i in a:
  print(i)
  b.append(glob.glob("Test_Features/"+i+"/Fake/*.npy"))

Lemon
Orange
Pear
Apple


In [22]:
count=0
for i in b:
  for j in i:
    if(count!=0):
      X=np.concatenate((X,np.load(j)))
    else:
      X=np.load(j)
    count+=1
    print(count)

In [62]:
trainX=np.load("Dobara.npy")
testX1=np.load("Dobara_test.npy")
testX2=np.load("Dobara_test1.npy")
trainy=np.zeros(trainX.shape[0])
testy1=np.zeros(testX1.shape[0])
testy2=np.ones(testX2.shape[0])
testX=np.concatenate((testX1,testX2))
testy=np.concatenate((testy1,testy2))
trainX.shape,testX.shape,trainy.shape,testy.shape

((2350, 1024), (2115, 1024), (2350,), (2115,))

In [64]:
# elliptic envelope for imbalanced classification
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.covariance import EllipticEnvelope
# generate dataset
# X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
# 	n_clusters_per_class=1, weights=[0.999], flip_y=0, random_state=4)
# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
# print(trainX.shape,trainy.shape)
# define outlier detection model
model = EllipticEnvelope(contamination=0.30)
# fit on majority class
trainX = trainX[trainy==0]
model.fit(trainX)
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
print('F1 Score: %.3f' % f1_score(testy, yhat, pos_label=-1))
print('Accuracy Percentage:', accuracy_score(testy, yhat)*100)

/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-33157.854225659204531 > -33159.016009791426768). You may want to try with a higher value of support_fraction (current value: 0.719).
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-33266.561479921598220 > -33275.238987055476173). You may want to try with a higher value of support_fraction (current value: 0.719).
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning:

F1 Score: 1.000
Accuracy Percentage: 100.0


In [68]:
# one-class svm for imbalanced binary classification
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM
# generate dataset
# X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
# 	n_clusters_per_class=1, weights=[0.999], flip_y=0, random_state=4)
# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
# define outlier detection model
model = OneClassSVM(gamma='scale', nu=0.30)
# fit on majority class
trainX = trainX[trainy==0]
model.fit(trainX)
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
print('F1 Score: %.3f' % f1_score(testy, yhat, pos_label=-1))
print('Accuracy Percentage:', accuracy_score(testy, yhat)*100)

F1 Score: 0.499
Accuracy Percentage: 33.2387706855792


In [69]:
# local outlier factor for imbalanced classification
from numpy import vstack
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neighbors import LocalOutlierFactor

# make a prediction with a lof model
def lof_predict(model, trainX, testX):
	# create one large dataset
	composite = vstack((trainX, testX))
	# make prediction on composite dataset
	yhat = model.fit_predict(composite)
	# return just the predictions on the test set
	return yhat[len(trainX):]

# # generate dataset
# X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
# 	n_clusters_per_class=1, weights=[0.999], flip_y=0, random_state=4)
# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
# define outlier detection model
model = LocalOutlierFactor(contamination=0.3)
# get examples for just the majority class
trainX = trainX[trainy==0]
# detect outliers in the test set
yhat = lof_predict(model, trainX, testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
print('F1 Score: %.3f' % f1_score(testy, yhat, pos_label=-1))
print('Accuracy Percentage:', accuracy_score(testy, yhat)*100)

F1 Score: 0.475
Accuracy Percentage: 31.15839243498818


In [70]:
# isolation forest for imbalanced classification
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import IsolationForest
# generate dataset
# X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
# 	n_clusters_per_class=1, weights=[0.999], flip_y=0, random_state=4)
# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
# # define outlier detection model
model = IsolationForest(contamination=0.3, behaviour='new')
# fit on majority class
trainX = trainX[trainy==0]
model.fit(trainX)
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
print('F1 Score: %.3f' % f1_score(testy, yhat, pos_label=-1))
print('Accuracy Percentage:', accuracy_score(testy, yhat)*100)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


F1 Score: 0.555
Accuracy Percentage: 38.43971631205674


In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(512, input_dim=2048, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    adagrad = Adam(lr=0.001, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])
    return model
model = create_model()
model.summary()

In [ ]:
def generator(train_indices, batch_size, itr, lines):
    # Create empty arrays to contain batch of features and labels#
    batch_images = []
    batch_labels = []
    startInd = itr * batch_size
    endInd = startInd + batch_size
    rand_exmp=np.random.permutation(len(lines))
    rand_exmp=rand_exmp[len(lines)-batch_size:]
    cc=0;
    batch_labels = np.zeros((batch_size, 4))
    for i in rand_exmp:
        line = lines[i].split('\t')
        image_path = os.path.join('', line[0])
        fmap = np.load(image_path)
        #print(fmap.shape)
        fmap1=np.sum(fmap,axis=0)
        norm1 = fmap1 / np.sum(fmap1)
        batch_images.append(norm1)
        batch_labels[cc,np.int(line[1])]=1
        cc=cc+1
    batch_images = np.array(batch_images, dtype=np.float32)
    batch_labels = np.array(batch_labels, dtype=np.int32)

    return batch_images, batch_labels


In [ ]:
import time
batch_size = 32 # increase batch size
nb_epoch = 20
lines = [line.rstrip('\n') for line in open('features.txt')]
indices = np.arange(len(lines))
train_indices = indices[:]  # split training
print(train_indices.shape)
iterations_t = np.int(len(train_indices) / batch_size)
t_loss_ep = []
for epoch in range(nb_epoch):
    ## training
    t_loss = []
    start = time.time()
    for i in range(iterations_t):
        X_train, y_train = generator(train_indices, batch_size, i, lines)
        history = model.fit(X_train, y_train, verbose=0)
        t_loss.append(history.history['loss'])
    stop = time.time()
    duration = stop - start
    t_loss_ep.append(np.mean(np.array(t_loss, dtype=np.float32)))
    print("Epoch: {}  Training Loss: {}  Time: {} sec".format(epoch, t_loss_ep[epoch], duration))
    print(epoch)
    model_name = "Saved Models/Model_" + str(epoch+1) + "Ground_Training_Epochs_b6.hdf5"
    model.save_weights(model_name)

In [ ]:
####### Testing
model.load_weights("Saved Models/Model_49Ground_Training_Epochs_b6.hdf5")
lines_testing = [line.rstrip('\n') for line in open('features_test.txt')]
Acc=np.zeros((4,1))
Matrixes=np.zeros((4,4))
for i in range(len(lines_testing)):
   # print(i)
    line = lines_testing[i].split('\t')
    image_path = os.path.join('', line[0])
    fmap = np.load(image_path)
    fmap1 = np.sum(fmap, axis=0)
    norm1 = fmap1 / np.linalg.norm(fmap1)
    #norm1=fmap1/fmap1.sum()
    input=np.zeros((1,2048))
    input[0]=norm1
    output = model.predict(input)
    sorted_indices = np.argsort(output[0])[::-1]
    GT=np.int(line[1])
    #print(sorted_indices[0])
    Matrixes[GT][sorted_indices[0]]=Matrixes[GT][sorted_indices[0]]+1
    if GT==sorted_indices[0]:
        Acc[GT]=Acc[GT]+1
print(Acc)
print(Acc.mean())
print("Accuracy is :",(sum(Acc)[0]/len(lines_testing))*100)